## DB N:N 실습문제

기존 movies_blog 프로젝트를 이어서 진행합니다.

Movie 리뷰에 좋아요를 할 수 있는 기능을 추가하려고 한다. 

아래 문제를 풀면서 기능을 구현하세요. 

#### 1. 여러 유저가 여러 리뷰에 좋아요를 누를 수 있도록 관계를 생성하세요( N:N )
1. 중계테이블명은 review_like_user 로 설정한다. 
2. 리뷰를 좋아요 누른 시간을 저장한다. 
3. 리뷰 목록에서 좋아요를 누를 수 있는 태그를 생성하세요. 
(이미 좋아요가 눌러져있다면 좋아요 취소로 바꿔주세요.)
4. 좋아요 태그 옆에 해당 리뷰가 받은 총 좋아요 개수를 출력하세요.
5. 로그인한 경우에만 좋아요가 가능하도록 하세요. 
6. 좋아요 태그 옆에 리뷰가 받은 좋아요 수를 클릭하면, 좋아요를 누른 유저 닉네임 리스트를 출력하는 기능을 구현하세요.

리뷰 목록에서 좋아요 버튼 사진 (좋아요 개수 포함)  
    <img src="practice_images/image72.jpg" width="30%" height="1%"/>

리뷰 목록에서 좋아요 취소 버튼 사진  
    <img src="practice_images/image73.jpg" width="30%" height="1%"/>

비로그인 시 좋아요 버튼이 없는 사진  
    <img src="practice_images/image74.jpg" width="30%" height="1%"/>

좋아요 태그를 눌렀을 때 이동한 좋아요 유저 목록 페이지 사진  
    <img src="practice_images/image75.jpg" width="30%" height="1%"/>

1. review 와 user 간의 관계를 형성하기 위해서 reviews/models.py에서 아래 클래스를 수정 및 추가하세요.
( reviews/models.py )

    ```python
    class Review(models.Model):
        movie_title = models.CharField(max_length=20)
        review_title = models.TextField()
        review_content = models.TextField()
        created_at = models.DateTimeField(auto_now_add=True)
        updated_at = models.DateTimeField(auto_now=True)
        user = models.ForeignKey(settings.AUTH_USER_MODEL, on_delete=models.CASCADE, related_name="reviews")
        like_users = models.ManyToManyField(settings.AUTH_USER_MODEL, related_name="like_reviews", through="ReviewLikeUser")

    class ReviewLikeUser(models.Model):
        review = models.ForeignKey(Review, on_delete=models.CASCADE)
        user = models.ForeignKey(settings.AUTH_USER_MODEL, on_delete=models.CASCADE)
        created_at = models.DateTimeField(auto_now_add=True)
        
        class Meta:
            db_table = "article_like_user"
    ```

2. 좋아요 버튼을 리뷰 목록에 출력하기 위해서 reviews/index.html을 수정

    ```html
        {% for review in reviews %}
          <li>
            <p>작성일 : 작성 시간 출력하기</p>
            <p>작성자 : {{ review.user }}</p>
            <p>영화 제목 : {{ review.movie_title }}</p>
            <a href="{% url "reviews:detail" review_idx=review.pk %}">리뷰 제목  : {{ review.review_title }}</a>
            ( {{ review.comments.all|length }} )
            <p>리뷰 내용: {{ review.review_content }}</p>
            <form action="{% url "reviews:like_review" review.pk %}" method="POST">
              {% csrf_token %}
              {% if request.user in review.like_users.all %}
              <input type="submit" value="좋아요 취소.">
              {% else %}
              <input type="submit" value="좋아요!">
              {% endif %}
            </form>
          </li>
          <hr>
        {% empty %}
          <li>작성된 리뷰가 없습니다.</li>
        {% endfor %}
    ```

3. index.html에서 필요한 reviews:like_review url을 생성
    ```python
    urlpatterns = [
    path('like_review/<int:review_idx>', views.like_review, name='like_review')  # 이하생략
    ]
    ```

4. urls.py에서 필요한 like_review 함수 생성하기
( reviews/views.py )

    ```python
    # reviews/views.py
    def like_review(request, review_idx):
        review = Review.objects.get(pk=review_idx)
        if review.like_users.filter(pk=request.user.id).exists():
            review.like_users.remove(request.user)
        else:
            review.like_users.add(request.user)
        return redirect('reviews:index')
    ```

5. 지금까지 받은 좋아요 개수를 반환하기 위해서 reviews/index.html 수정 
    
    ```html
        <p>작성일 : 작성 시간 출력하기</p>
        <p>작성자 : {{ review.user }}</p>
        <p>영화 제목 : {{ review.movie_title }}</p>
        <a href="{% url "reviews:detail" review_idx=review.pk %}">리뷰 제목  : {{ review.review_title }}</a>
        ( {{ review.comments.all|length }} )
        <p>리뷰 내용: {{ review.review_content }}</p>
        <p>좋아요: {{ review.like_users.all|length }}</p>   
    ```

6. 로그인하지 않은 경우에는 좋아요가 보이지 않게 하기 위하여 reviews/index.html 수정 

    ```html
        <p>좋아요: {{ review.like_users.all|length }}</p>
        {% if user.is_authenticated %}
            <form action="{% url "reviews:like_review" review.pk %}" method="POST">
            {% csrf_token %}
            {% if request.user in review.like_users.all %}
            <input type="submit" value="좋아요 취소.">
            {% else %}
            <input type="submit" value="좋아요!">
            {% endif %}
            </form>
        {% endif %}
    ```

7. 내부적으로도 로그인을 하지 않은 유저는 좋아요를 할 수 없도록 수정
( reviews/views.py 의 like_review 함수 수정)

    ```python
    # reviews/views.py
    @login_required
    def like_review(request, review_idx):
        review = Review.objects.get(pk=review_idx)
        if review.like_users.filter(pk=request.user.id).exists():
            review.like_users.remove(request.user)
        else:
            review.like_users.add(request.user)
        return redirect('reviews:index')
    ```

8. 좋아요 수를 누를 경우에 좋아요를 누른 유저목록을 보여주는 페이지로 이동하기 위해 reviews/index.html 수정하기

    ```html
        <p>리뷰 내용: {{ review.review_content }}</p>
        <p>좋아요: <a href="{% url "reviews:review_like_users" review.pk %}">{{ review.like_users.all|length }}</a></p>
    ```

9. review_like_users url을 생성 
( reviews/urls.py 에 아래 코드 추가 )

    ```python
    
urlpatterns = [
    path('review_like_users/<int:review_idx>', views.review_like_users, name='review_like_users')  # 이하 생략
    ]
    ```

10. reviews/views.py 에 review_like_users 함수 추가
    ```python
    # reviews/views.py
    def review_like_users(request, review_idx):
        review = Review.objects.get(pk=review_idx)
        like_users = review.like_users.all()
        context = {
            'like_users': like_users,
            'review': review
        }
        return render(request, 'reviews/review_like_users.html', context)
    ```

11. review_like_users 함수가 랜더링하는 review_like_users.html 생성
    ```html
        {% extends "base.html" %}

        {% block content %}
        <h2>[ {{ review.review_title}} ]리뷰 좋아요 유저 목록</h2>
        <ul>
        {% for like_user in like_users %}
            <li>{{ like_user.username }} 님이 좋아요를 눌렀습니다.</li>
        {% endfor %}
        </ul>
        {% endblock content %}
    ```


#### 2. 팔로우 기능 완성하세요. 
1. localhost:8000/movies/ 페이지에 유저 목록을 출력한다.

2. 해당 유저를 클릭하면 유저 상세 페이지로 이동해지도록 한다. 이 때, 현재 유저 상세 페이지는 로그인한 유저의 상세 페이지로 이동하는데, variable routing을 이용해서 다른 유저의 id를 통한 페이지 이동이 되도록 구현한다. 
또한, 본인이 아니라면 정보 수정/로그아웃/회원탈퇴할 수 없도록 한다.

3. 유저 디테일 페이지에 팔로우 기능을 추가한다.
4. 유저 디테일 페이지에 현재 팔로우 유저와 팔로잉 유저 수를 출력한다.
5. 본인 스스로 팔로우할 수 없어야한다.

localhost:8000/movies/ 페이지 사진  
    <img src="practice_images/image76.jpg" width="30%" height="1%"/>

비로그인상태에서 1번 회원 페이지에 들어갔을 때 사진  
    <img src="practice_images/image77.jpg" width="30%" height="1%"/>

로그인 후 본인 회원 페이지에 들어갔을 때 사진  
    <img src="practice_images/image78.jpg" width="30%" height="1%"/>

다른 유저를 언팔로우 했을 떄의 사진   
    <img src="practice_images/image79.jpg" width="30%" height="1%"/>

다른 유저를 팔로우 했을 떄의 사진   
    <img src="practice_images/image80.jpg" width="30%" height="1%"/>

1. 메인페이지에서 유저 목록을 출력하기 위해서 movies/views.py 의 index 함수에서 유저를 조회하는 로직을 추가한다.

    ```python
    def index(request):
        # movie = request.GET.get('movie_pk')
        movie_list = Movie.objects.all()
        user_list = get_user_model().objects.all()
        context = {
            # 'movie': movie_pk
            'movie_list': movie_list,
            'user_list':user_list
        }
        return render(request, 'movies/index.html', context)
    ```

2. movies/index.html에서 유저 목록 및 수를 출력하는 코드를 작성한다.

    ```html
        <a href="/movies/create_movie/">영화 추가</a>

        <h4>현재 유저 목록 (유저 수: {{ user_list|length }} ) </h4>
        <ul>
        {% for user in user_list %}
            <li><a href="{% url "accounts:index" user.pk %}">{{ user.username }}</a></li>
        {% endfor %}
        </ul>
    ```

3. 수정된 profile url에 따라 accounts/urls.py의 profile url을 수정한다.
    ```python
    urlpatterns = [
    path('index/<int:user_idx>', views.index, name='index')  # 이하생략
    ]
    ```

4. accounts/views.py 의 index 함수를 수정한다.

    ```python
    # accounts/views.py
    from django.contrib.auth.decorators import login_required
    from .models import User

    @login_required
    def index(request, user_idx):
        user = User.objects.get(pk=user_idx)
        reviews = user.reviews.all()
        comments = user.comments.all()
        context = {
            'reviews': reviews,
            'comments': comments,
            'target_user': user
        }
        return render(request, "accounts/index.html", context)
    ```

5. base.html 에서도 accounts:index url을 사용함으로 수정해준다.
    ```html
      <a href="{% url "accounts:index" user.pk %}">[PROFILE]</a> |
    ```

6. 현재 프로필 페이지에서는 django 가 자동으로 입력해주는 user를 사용하고 있어서, 로그인한 유저의 정보가 출력되고 있기 떄문에 user 변수를 index 함수에서 반환해주고 있는 target_user로 바꿔준다.  
또한, request.user 와 target_user를 비교해 자신의 프로필인 경우에만 수정할 수 있도록 한다.  
( accounts/index.html )  

    ```html
    <!-- accounts/index.html-->
    {% extends "base.html" %}
    {% block content %}
    <h1>회원정보 페이지</h1>
    <hr>
    <h4>계정이름: {{ target_user.username}}</h4>
    <h4>이메일: {{ target_user.email}}</h4>
    <hr>
    <h2>작성한 리뷰 목록</h2>
    {% if reviews %}
    <ol>
        {% for review in reviews %}
            <li><a href="{% url "reviews:detail" review.pk %}">리뷰 제목: {{ review.review_title}}</a>
            </li>
        {% endfor %}
    </ol>
    {% else %}
        <p>작성한 리뷰가 없습니다.</p>
    {% endif %}
    <hr>
    <h2>작성한 댓글 목록</h2>
    {% if comments %}
    <ol>
        {% for comment in comments %}
            <li><a href="{% url "reviews:detail" comment.review.id %}">댓글: {{ comment.content}}</a>
            </li>
        {% endfor %}
    </ol>
    {% else %}
        <p>작성한 댓글이 없습니다.</p>
    {% endif %}
    <hr>
    {% if user == target_user %}
        <form action="{% url "accounts:update" %}" mehtod="POST">
            {% csrf_token %}
            <input type="submit" value="회원정보 수정하기">
        </form><br>
        <form action="{% url "accounts:logout" %}" mehtod="POST">
            {% csrf_token %}
            <input type="submit" value="로그아웃하기">
        </form><br>
        <form action="{% url "accounts:delete" %}" mehtod="POST">
            {% csrf_token %}
            <input type="submit" value="회원탈퇴하기">
        </form><br>
    {% endif %}
    {% endblock content %}
    ```

6.1  ctrl+ shift + F 를 눌러 'accounts:index' 를 검색한 후에 'accounts:index' 경로로 향하는 모든 코드에 user.id를 파라미터로 전달한다.


7. 유저간이 팔로우를 위해서 N:N 모델링을 한다. 
( accounts/models.py)

    ```python
    class User(AbstractUser):
        followings = models.ManyToManyField('self', symmetrical=False, related_name='followers')
    ```

8. 팔로우 기능을 구현할 url을 생성한다.
( accounts/urls.py)
    
    ```python
        urlpatterns = [
            path('follow/<int:user_idx>', views.follow, name='follow')
        ]
    ```

9. follow 함수를 구현한다.
( accounts/views.py에 follow 함수 생성)

    ```python
    # accounts/views.py
    def follow(request, user_idx):
        user = get_user_model().objects.get(pk=user_idx)
        if request.user == user:
            return redirect('accounts:index', user.id)
        
        if request.user in user.followers.all():
            user.followers.remove(request.user)
        else:
            user.followers.add(request.user)
        return redirect('accounts:index', user.id)
    ```

10. 회원 정보 페이지에 팔로우 버튼을 구현하고, 팔로우 중인 경우에는 언팔로우가 나타나도록 한다. 또 팔로우/팔로잉 수를 출력하도록 한다. 또, 본인 스스로는 팔로우할 수 없도록 한다. 

( accounts/index.html ) 

    ```html
    <h4>팔로워: {{ target_user.followers.all|length }} </h4>
    <h4>팔로잉: {{ target_user.followings.all|length }} </h4>
    {% if target_user != request.user %}
    <form action="{% url "accounts:follow" target_user.pk %}" method="POST">
        {% csrf_token %}
        {% if request.user in target_user.followers.all %}
        <input type="submit" value="언팔로우">
        {% else %}
        <input type="submit" value="팔로우">
        {% endif %}
    </form>
    {% endif %}
    ```
